<a href="https://colab.research.google.com/github/DesiPilla/frus-event-exctraction/blob/master/MongoDB_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import json
import pandas as pd
from pymongo import MongoClient

In [13]:
cd '/content/drive/My Drive/FRUS/Raw Data/Combined Data'

/content/drive/.shortcut-targets-by-id/119/FRUS/Raw Data


**SSH into the MongoDB located on the EC2 instance**

In [24]:
client = MongoClient('mongodb://52.71.228.156:27017/')
client.list_database_names()

['admin', 'config', 'local']

**Select or create a database named `FRUS`**

In [28]:
db = client.FRUS
db.list_collection_names()

[]

**Take the preprared data and convert it to a JSON file**

In [2]:
def prepare_collection(path):
    df = pd.read_csv(path).rename(columns={'website':'source', 'text':'context'})
    df['stanford'] = 0
    return df.to_dict('records')

In [18]:
taft_collection = prepare_collection('taft_df.csv')
taft_collection[:2]

[{'Unnamed: 0': 106432,
  'context': 'The Ambassador in France ( Herrick ) to the Secretary of State American Embassy , Paris , July 28, 1914, 4 p.m. [ Received 7:30 p.m. ] [Telegram] To be communicated to the President: Situation in Europe is regarded here as the gravest in history. It is apprehended that civilization is threatened by demoralization which would follow a general conflagration. Demonstrations made against war here last night by laboring classes; it is said to be the first instance of its kind in France. It is felt that if Germany once mobilizes no backward step will be taken. France has strong reliance on her army but it is not giving way to undue excitement. There is faith and reliance on our high ideals and purposes, so that I believe expression from our nation would have great weight in this crisis. My opinion is encouraged at reception given utterances of British Minister for Foreign Affairs. I believe that a strong plea for delay and moderation from the President o

**Add the JSON file to the MongoDB as a collection**

In [21]:
db.Taft.insert_many(taft_collection, ordered=False)
print("Data has been exported to MongoDB server.")

Data has been exported to MongoDB server.


In [20]:
db.list_collection_names()

['taft']